# 🎮 AI Chess - Generate Training Data

## Tạo Dữ Liệu Training trên Google Colab

Notebook này sẽ:
1. Cài đặt các thư viện cần thiết
2. Copy code agents từ project
3. Chạy Minimax self-play để tạo data
4. Lưu file `chess_data.csv` để download

---

**⚠️ Lưu ý:** Notebook này chạy trên Google Colab (miễn phí GPU) để tạo data nhanh hơn.

## 1. Cài Đặt Thư Viện

In [ ]:
# Cài đặt python-chess và tqdm
!pip install python-chess tqdm -q

print("✓ Đã cài đặt các thư viện cần thiết")

## 2. Import Libraries

In [ ]:
import chess
import csv
import random
from tqdm import tqdm
import numpy as np

print(f"✓ Libraries imported successfully")
print(f"   python-chess version: {chess.__version__}")

## 3. Utils Functions

Copy các hàm từ `utils.py`

In [ ]:
# ============================================================
# UTILS FUNCTIONS (from utils.py)
# ============================================================

def get_piece_value(piece):
    """Trả về giá trị của quân cờ"""
    values = {
        chess.PAWN: 100,
        chess.KNIGHT: 320,
        chess.BISHOP: 330,
        chess.ROOK: 500,
        chess.QUEEN: 900,
        chess.KING: 20000
    }
    return values.get(piece.piece_type, 0)


def get_position_value(piece, square, endgame=False):
    """Trả về giá trị vị trí của quân cờ"""
    # Position tables (simplified)
    pawn_table = [
        0,  0,  0,  0,  0,  0,  0,  0,
        50, 50, 50, 50, 50, 50, 50, 50,
        10, 10, 20, 30, 30, 20, 10, 10,
        5,  5, 10, 25, 25, 10,  5,  5,
        0,  0,  0, 20, 20,  0,  0,  0,
        5, -5,-10,  0,  0,-10, -5,  5,
        5, 10, 10,-20,-20, 10, 10,  5,
        0,  0,  0,  0,  0,  0,  0,  0
    ]
    
    knight_table = [
        -50,-40,-30,-30,-30,-30,-40,-50,
        -40,-20,  0,  0,  0,  0,-20,-40,
        -30,  0, 10, 15, 15, 10,  0,-30,
        -30,  5, 15, 20, 20, 15,  5,-30,
        -30,  0, 15, 20, 20, 15,  0,-30,
        -30,  5, 10, 15, 15, 10,  5,-30,
        -40,-20,  0,  5,  5,  0,-20,-40,
        -50,-40,-30,-30,-30,-30,-40,-50
    ]
    
    king_table_endgame = [
        -50,-40,-30,-20,-20,-30,-40,-50,
        -30,-20,-10,  0,  0,-10,-20,-30,
        -30,-10, 20, 30, 30, 20,-10,-30,
        -30,-10, 30, 40, 40, 30,-10,-30,
        -30,-10, 30, 40, 40, 30,-10,-30,
        -30,-10, 20, 30, 30, 20,-10,-30,
        -30,-30,  0,  0,  0,  0,-30,-30,
        -50,-30,-30,-30,-30,-30,-30,-50
    ]
    
    # Lấy index cho bảng (0-63)
    sq_index = square
    if piece.color == chess.BLACK:
        sq_index = 63 - square  # Mirror cho quân đen
    
    if piece.piece_type == chess.PAWN:
        return pawn_table[sq_index]
    elif piece.piece_type == chess.KNIGHT:
        return knight_table[sq_index]
    elif piece.piece_type == chess.KING and endgame:
        return king_table_endgame[sq_index]
    
    return 0


def is_endgame(board):
    """Kiểm tra xem có phải endgame không"""
    # Đếm số quân (không tính Vua và Tốt)
    queens = len(board.pieces(chess.QUEEN, chess.WHITE)) + len(board.pieces(chess.QUEEN, chess.BLACK))
    rooks = len(board.pieces(chess.ROOK, chess.WHITE)) + len(board.pieces(chess.ROOK, chess.BLACK))
    minors = len(board.pieces(chess.KNIGHT, chess.WHITE)) + len(board.pieces(chess.KNIGHT, chess.BLACK)) + \
             len(board.pieces(chess.BISHOP, chess.WHITE)) + len(board.pieces(chess.BISHOP, chess.BLACK))
    
    # Endgame nếu:
    # - Không còn Hậu
    # - Hoặc mỗi bên chỉ còn 1 Hậu và không có quân khác
    # - Hoặc tổng quân nhỏ <= 6
    if queens == 0:
        return True
    if queens == 2 and rooks == 0 and minors == 0:
        return True
    if queens + rooks + minors <= 6:
        return True
    
    return False


print("✓ Utils functions loaded")

## 4. Minimax Agent

Simplified version - chỉ cần evaluation và move ordering

In [ ]:
# ============================================================
# MINIMAX AGENT (Simplified for Data Generation)
# ============================================================

class MinimaxAgent:
    """Simplified Minimax Agent for data generation"""
    
    def __init__(self, depth=2):
        self.name = "Minimax Agent"
        self.depth = depth
        self.nodes_searched = 0
    
    def reset_stats(self):
        self.nodes_searched = 0
    
    def order_moves(self, board, moves):
        """Sắp xếp nước đi (MVV-LVA)"""
        move_scores = []
        
        for move in moves:
            score = 0
            
            if board.is_capture(move):
                captured = board.piece_at(move.to_square)
                moving = board.piece_at(move.from_square)
                if captured and moving:
                    score += 1000 + get_piece_value(captured) - get_piece_value(moving) // 10
            
            if move.promotion:
                score += 900
            
            move_scores.append((score, move))
        
        move_scores.sort(reverse=True, key=lambda x: x[0])
        return [move for _, move in move_scores]
    
    def evaluate_board(self, board):
        """Đánh giá bàn cờ (simplified version)"""
        if board.is_checkmate():
            return -999999 if board.turn == chess.WHITE else 999999
        
        if board.is_stalemate() or board.is_insufficient_material():
            return 0
        
        score = 0
        endgame = is_endgame(board)
        
        # Material + Position
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece:
                piece_value = get_piece_value(piece)
                position_value = get_position_value(piece, square, endgame)
                total = piece_value + position_value
                
                if piece.color == chess.WHITE:
                    score += total
                else:
                    score -= total
        
        return score
    
    def minimax(self, board, depth, alpha, beta, maximizing):
        """Minimax with alpha-beta pruning"""
        self.nodes_searched += 1
        
        if depth == 0 or board.is_game_over():
            return self.evaluate_board(board)
        
        legal_moves = list(board.legal_moves)
        ordered_moves = self.order_moves(board, legal_moves)
        
        if maximizing:
            max_eval = float('-inf')
            for move in ordered_moves:
                board.push(move)
                eval_score = self.minimax(board, depth - 1, alpha, beta, False)
                board.pop()
                max_eval = max(max_eval, eval_score)
                alpha = max(alpha, eval_score)
                if beta <= alpha:
                    break
            return max_eval
        else:
            min_eval = float('inf')
            for move in ordered_moves:
                board.push(move)
                eval_score = self.minimax(board, depth - 1, alpha, beta, True)
                board.pop()
                min_eval = min(min_eval, eval_score)
                beta = min(beta, eval_score)
                if beta <= alpha:
                    break
            return min_eval
    
    def get_move(self, board):
        """Tìm nước đi tốt nhất"""
        self.reset_stats()
        
        legal_moves = list(board.legal_moves)
        if not legal_moves:
            return None
        
        best_move = None
        best_value = float('-inf') if board.turn == chess.WHITE else float('inf')
        alpha = float('-inf')
        beta = float('inf')
        
        ordered_moves = self.order_moves(board, legal_moves)
        
        for move in ordered_moves:
            board.push(move)
            
            if board.turn == chess.BLACK:
                move_value = self.minimax(board, self.depth - 1, alpha, beta, False)
            else:
                move_value = self.minimax(board, self.depth - 1, alpha, beta, True)
            
            board.pop()
            
            if board.turn == chess.WHITE:
                if move_value > best_value:
                    best_value = move_value
                    best_move = move
                    alpha = max(alpha, move_value)
            else:
                if move_value < best_value:
                    best_value = move_value
                    best_move = move
                    beta = min(beta, move_value)
        
        return best_move


print("✓ MinimaxAgent loaded")

# Quick test
agent = MinimaxAgent(depth=2)
board = chess.Board()
move = agent.get_move(board)
print(f"✓ Test move: {move} (searched {agent.nodes_searched} nodes)")

## 5. Data Generation Function

In [ ]:
# ============================================================
# DATA GENERATION
# ============================================================

def generate_game_data(num_games=100, depth=2, save_interval=100):
    """
    Tạo dữ liệu từ Minimax self-play
    
    Args:
        num_games: Số ván cờ
        depth: Độ sâu minimax
        save_interval: Backup sau mỗi N games
    
    Returns:
        List of (fen, score) tuples
    """
    data = []
    agent = MinimaxAgent(depth=depth)
    
    print(f"\n🎮 Generating data from {num_games} games (depth={depth})")
    print(f"📊 Expected: ~{num_games * 40} positions")
    print(f"💾 Auto-save every {save_interval} games\n")
    
    for game_num in tqdm(range(num_games), desc="Games"):
        board = chess.Board()
        move_count = 0
        max_moves = 150
        
        while not board.is_game_over() and move_count < max_moves:
            # Lưu trạng thái
            fen = board.fen()
            score = agent.evaluate_board(board)
            data.append((fen, score))
            
            # Lấy nước đi
            move = agent.get_move(board)
            if move is None:
                break
            
            board.push(move)
            move_count += 1
            
            # Thêm random move (15% chance) để đa dạng
            if random.random() < 0.15:
                legal_moves = list(board.legal_moves)
                if legal_moves and not board.is_game_over():
                    random_move = random.choice(legal_moves)
                    board.push(random_move)
                    move_count += 1
        
        # Backup định kỳ
        if (game_num + 1) % save_interval == 0:
            print(f"\n💾 Backup: {len(data)} positions after {game_num+1} games")
    
    print(f"\n✓ Generated {len(data):,} positions from {num_games} games")
    print(f"  Average: {len(data)/num_games:.1f} positions/game")
    
    return data


def save_to_csv(data, filename='chess_data.csv'):
    """Lưu data vào CSV"""
    print(f"\n💾 Saving to {filename}...")
    
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['fen', 'score'])  # Header
        writer.writerows(data)
    
    print(f"✓ Saved {len(data):,} positions")
    print(f"  File size: {len(data) * 100 / 1024:.1f} KB")


print("✓ Data generation functions loaded")

## 6. ⚙️ Configuration

Chọn số lượng games để generate

In [ ]:
# ============================================================
# CONFIGURATION
# ============================================================

print("📊 Recommended configurations:")
print("  - 100 games:   ~5 min,  ~4,000 positions (quick test)")
print("  - 500 games:   ~25 min, ~20,000 positions (good)")
print("  - 1000 games:  ~50 min, ~40,000 positions (recommended)")
print("  - 2000 games:  ~100 min, ~80,000 positions (excellent)")
print("\n⚡ Google Colab is much faster than local machine!\n")

# Configuration
NUM_GAMES = 1000  # Thay đổi số này
DEPTH = 2         # Depth 2 cho cân bằng speed/quality
SAVE_INTERVAL = 100  # Backup every 100 games

print(f"✓ Configuration:")
print(f"  NUM_GAMES = {NUM_GAMES}")
print(f"  DEPTH = {DEPTH}")
print(f"  SAVE_INTERVAL = {SAVE_INTERVAL}")

## 7. 🚀 Generate Data

**Chạy cell này để bắt đầu generate data**

In [ ]:
# ============================================================
# GENERATE DATA
# ============================================================

print("="*60)
print("🎮 AI CHESS - DATA GENERATION")
print("="*60)

# Generate
data = generate_game_data(
    num_games=NUM_GAMES,
    depth=DEPTH,
    save_interval=SAVE_INTERVAL
)

# Save to CSV
save_to_csv(data, filename='chess_data.csv')

print("\n" + "="*60)
print("✅ GENERATION COMPLETED!")
print("="*60)
print(f"\n📈 Statistics:")
print(f"  Total positions: {len(data):,}")
print(f"  Average: {len(data)/NUM_GAMES:.1f} positions/game")
print(f"  File: chess_data.csv")
print("\n🎯 Next steps:")
print("  1. Download 'chess_data.csv' from Colab")
print("  2. Continue with train_model.ipynb to train ML model")
print("="*60)

## 8. 📊 Data Analysis (Optional)

In [ ]:
# ============================================================
# DATA ANALYSIS
# ============================================================

import pandas as pd
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv('chess_data.csv')

print("📊 Data Statistics:\n")
print(df.describe())

# Plot score distribution
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(df['score'], bins=50, edgecolor='black')
plt.title('Score Distribution')
plt.xlabel('Score')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.boxplot(df['score'])
plt.title('Score Boxplot')
plt.ylabel('Score')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✓ Data looks good!")
print(f"  Mean score: {df['score'].mean():.2f}")
print(f"  Std score: {df['score'].std():.2f}")
print(f"  Min: {df['score'].min():.0f}, Max: {df['score'].max():.0f}")

## 9. 💾 Download File

Download `chess_data.csv` về máy

In [ ]:
# Download file
from google.colab import files

print("📥 Downloading chess_data.csv...")
files.download('chess_data.csv')
print("✓ Download started! Check your browser downloads.")

---

## 🎉 Hoàn Tất!

**Bước tiếp theo:**
1. ✅ Download `chess_data.csv`
2. ⏭️ Chạy `train_model.ipynb` để train model
3. 📦 Download `chess_model.h5` và `normalization_params.npy`
4. 🚀 Test ML Agent trong game!

---

**Tips:**
- Colab free tier: ~12 giờ runtime
- 1000 games ≈ 50 phút trên Colab
- Có thể tăng lên 2000-3000 games cho model tốt hơn
- Depth=2 là optimal (balance speed vs quality)

---